# Uniform Triadic Transformations

The implementation of Uniform Triadic Transformations.

Referenses

Ada Zhang, *The Framework of Music Theory as Represented with Groups*, https://sites.math.washington.edu/~morrow/336_09/papers/Ada.pdf, 2009

In [1]:
using MIDI

In [2]:
struct PitchClass
    note
    PitchClass(n=0) = new(n)
end

In [3]:
import Base.+
+(p::PitchClass, t::Integer)    = PitchClass(mod(p.note+t, 12))
+(t::Integer, p::PitchClass)    = PitchClass(mod(p.note+t, 12))
+(p::PitchClass, t::PitchClass) = PitchClass(mod(p.note+t.note, 12))

+ (generic function with 168 methods)

In [4]:
import Base.-
-(p::PitchClass)                = PitchClass(mod(-p.note, 12))
-(p::PitchClass, t::Integer)    = PitchClass(mod(p.note-t, 12))
-(t::Integer, p::PitchClass)    = PitchClass(mod(t-p.note, 12))
-(p::PitchClass, t::PitchClass) = PitchClass(mod(p.note-t.note, 12))

- (generic function with 181 methods)

In [5]:
# testing
p = PitchClass(1)
t = PitchClass(2)
p + t

PitchClass(3)

In [6]:
@enum Mode Major Minor

# Triad
struct Triad
    root
    sign
    Triad(root = 0, sign = Major) = new(PitchClass(root), sign)
    Triad(root::PitchClass, sign = Major) = new(root, sign)
end

In [7]:
import Base.*

function *(l::Mode, r::Mode)
    Mode(mod(Int(l)+Int(r), 2))
end

function *(l::Triad, r::Triad)
    root = l.root + r.root
    sign = l.sign * r.sign
    Triad(root, sign)
end

* (generic function with 345 methods)

In [8]:
Triad(2, Major)*Triad(5, Minor)

Triad(PitchClass(7), Minor::Mode = 1)

In [9]:
# Gamma-Interval
function int(l::Triad, r::Triad)
    root = r.root - l.root
    sign = l.sign * r.sign
    Triad(root, sign)
end

int (generic function with 1 method)

In [10]:
int(Triad(2, Major), Triad(5, Minor))

Triad(PitchClass(3), Minor::Mode = 1)

In [11]:
struct UniformTriadicTransformation
    sign
    tp
    tm
    UniformTriadicTransformation(sign = Major, tp = 0, tm = 0) = new(sign, tp, tm)
end

In [12]:
const UTT = UniformTriadicTransformation

UniformTriadicTransformation

In [13]:
function (u::UTT)(tr::Triad)
    t = (1-Int(tr.sign))*u.tp + Int(tr.sign)*u.tm
    s = u.sign * tr.sign
    Triad(tr.root + t, s)
end

In [14]:
u = UTT(Minor, 2, 3)
@show u(Triad(2, Major))
@show u(Triad(10, Minor))

u(Triad(2, Major)) = Triad(PitchClass(4), Minor::Mode = 1)
u(Triad(10, Minor)) = Triad(PitchClass(1), Major::Mode = 0)


Triad(PitchClass(1), Major::Mode = 0)

In [15]:
function *(u::UTT, v::UTT)
    s  = u.sign * v.sign
    tp = u.tp + (1-Int(u.sign))*v.tp + Int(u.sign)*v.tm
    tm = u.tm + Int(u.sign)*v.tp + (1-Int(u.sign))*v.tm
    UTT(s, tp, tm)
end

* (generic function with 346 methods)

In [16]:
u = UTT(Minor, 9, 8)
t = Triad()
for i in 1:24
    tn = u(t)
    println(t)
    t = tn
end

Triad(PitchClass(0), Major::Mode = 0)
Triad(PitchClass(9), Minor::Mode = 1)
Triad(PitchClass(5), Major::Mode = 0)
Triad(PitchClass(2), Minor::Mode = 1)
Triad(PitchClass(10), Major::Mode = 0)
Triad(PitchClass(7), Minor::Mode = 1)
Triad(PitchClass(3), Major::Mode = 0)
Triad(PitchClass(0), Minor::Mode = 1)
Triad(PitchClass(8), Major::Mode = 0)
Triad(PitchClass(5), Minor::Mode = 1)
Triad(PitchClass(1), Major::Mode = 0)
Triad(PitchClass(10), Minor::Mode = 1)
Triad(PitchClass(6), Major::Mode = 0)
Triad(PitchClass(3), Minor::Mode = 1)
Triad(PitchClass(11), Major::Mode = 0)
Triad(PitchClass(8), Minor::Mode = 1)
Triad(PitchClass(4), Major::Mode = 0)
Triad(PitchClass(1), Minor::Mode = 1)
Triad(PitchClass(9), Major::Mode = 0)
Triad(PitchClass(6), Minor::Mode = 1)
Triad(PitchClass(2), Major::Mode = 0)
Triad(PitchClass(11), Minor::Mode = 1)
Triad(PitchClass(7), Major::Mode = 0)
Triad(PitchClass(4), Minor::Mode = 1)


In [17]:
function RiemannianTriadTransformation(sign, t)
    UTT(sign, t, -t)
end

RiemannianTriadTransformation (generic function with 1 method)

In [18]:
const RTT = RiemannianTriadTransformation

RiemannianTriadTransformation (generic function with 1 method)

In [19]:
# to midi
length = 192*8
file = MIDIFile()
track = MIDITrack()
notes = Notes()

for i in 0:24
    third, fifth = t.sign == Major ? (4, 7) : (3, 7)
    push!(notes, Note(t.root.note + 60,         100, i*length, length))
    push!(notes, Note(t.root.note + 60 + third, 100, i*length, length))
    push!(notes, Note(t.root.note + 60 + fifth, 100, i*length, length))
    t = u(t)
end

addnotes!(track, notes)
addtrackname!(track, "UUTs")
push!(file.tracks, track)
writeMIDIFile("test.mid", file)

MIDIFile:
  format: 0
  tracks: 1
  tpq: 960